# Model Serving Function

In [1]:
import mlrun
import os

In [2]:
import numpy as np
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [3]:
# nuclio: end-code

## Deploy and Test The Function

If you would like to change the location of the source data, set the `SAMPLE_DATA_SOURCE_URL_PREFIX` environment variable.

For example, set it to `/v3io/projects/demos-data/iguazio/`

In [4]:
# Set the source-data URL
url_prefix = os.environ.get('SAMPLE_DATA_SOURCE_URL_PREFIX', 'https://s3.wasabisys.com/iguazio/')

In [5]:
models_path = f'{url_prefix.rstrip("/")}/models/lightgbm/SampleModel.pkl'

In [6]:
fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'edmondg', 'framework': 'lightgbm'},
                            code_output='.',
                            image='mlrun/mlrun',
                            kind='serving')
fn.spec.build.commands = ['pip install lightgbm']
fn.spec.default_class = 'LGBMModel'

In [7]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [8]:
# deploy the function
fn.apply(mlrun.platforms.auto_mount())
address = fn.deploy()

> 2021-10-05 06:15:37,500 [info] Starting remote function deploy
2021-10-05 06:15:37  (info) Deploying function
2021-10-05 06:15:37  (info) Building
2021-10-05 06:15:37  (info) Staging files and preparing base images
2021-10-05 06:15:37  (info) Building processor image
2021-10-05 06:15:50  (info) Build complete
2021-10-05 06:16:06  (info) Function deploy complete
> 2021-10-05 06:16:06,365 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-lightgbm-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev8.lab.iguazeng.com:30969']}


In [9]:
# test the function
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

> 2021-10-05 06:16:12,667 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-default-lightgbm-serving.default-tenant.svc.cluster.local:8080/v2/models/nyc-taxi-server/predict'}


{'id': '5011cfe1-602b-433d-bd99-3174c77b6ac7',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}